# Yelp Analysis

In [1]:
import numpy as np
import pandas as pd

In [2]:
# ---- Importing Data ---- #

path = 'yelp_csv\yelp_academic_dataset_user.csv'
df = pd.read_csv(path)
    
business_file_path = 'yelp_csv\yelp_academic_dataset_business.csv'
business_df = pd.read_csv(business_file_path)

review_file_path = 'yelp_csv\yelp_academic_dataset_review.csv'
reviews = pd.read_csv(review_file_path)


C:\Users\Oliver\AppData\Local\Temp\ipykernel_13472\1180441733.py:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\Oliver\AppData\Local\Temp\ipykernel_13472\1180441733.py:7: DtypeWarning: Columns (27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  business_df = pd.read_csv(business_file_path)


### Dashboard Data Preprocessing

In [4]:
# ---- Business Metrics ---- #

categories = set()

clean_bdf = business_df.dropna(subset=['categories'])
clean_bdf.loc[:, 'categories'] = clean_bdf['categories'].apply(lambda x: x.split(', '))

clean_bdf['categories'].apply(categories.update)

exploded = clean_bdf.explode('categories')

exploded.to_csv('out/business_cleaned.csv')

In [4]:
elites = df.dropna(subset=['elite'])

df['elite_status'] = df['elite'].apply(lambda x: isinstance(x, str))

output = pd.DataFrame()

output['elite'] = df['elite'].apply(lambda x: isinstance(x, str))

reviews['review_length'] = reviews['text'].apply(len)

In [19]:
# ---- User Metrics ---- #

mean_review_lengths = reviews.groupby('user_id')['review_length'].mean().reset_index()

output = df.merge(mean_review_lengths, on='user_id', how='left')


compliment_cols = [x for x in df.columns if x.startswith('compliment')]

compliments = df[compliment_cols]

output.insert(0, 'top_compliment_type', compliments.idxmax('columns', numeric_only=True))

wanted_columns = ['user_id', 'top_compliment_type', 'elite_status', 'review_count', 'review_length', 'average_stars', 'fans']

output = output[wanted_columns]
output.to_csv('out/user_data.csv')


In [20]:
# ---- Review Metrics ---- #
reviews = reviews.merge(df[['user_id', 'elite_status']], on='user_id', how='left')
reviews_reduced = reviews[['elite_status', 'date', 'useful']]

reviews_reduced.to_csv('out/reviews_reduced.csv')